# Flame

In order to download the models we need to look at: https://flame.is.tue.mpg.de/download.php
This could be usefull if onw want's to look how to load the FLAME model from the SMPL loader: https://github.com/Rubikplayer/flame-fitting/blob/master/smpl_webuser/serialization.py#L117
Useful utils if one needs to transform the chumpy format into nupy or torch: https://github.com/vchoutas/smplx/blob/main/smplx/utils.py

NOTE: That if one want't to unpickle old python=2.x numpy code, we need to use the encoding="latin1". For more information please refere to: https://docs.python.org/3/library/pickle.html


In [ ]:
import pickle
from pathlib import Path
import numpy as np
import torch
import warnings


def to_tensor(array, dtype=torch.float32):
    if torch.is_tensor(array):
        return array
    return torch.tensor(array, dtype=dtype)


def to_np(array, dtype=np.float32):
    if "scipy.sparse" in str(type(array)):
        array = array.todense()
    return np.array(array, dtype=dtype)


def load_flame_no_jaw(flame_dir: str | Path):
    # https://github.com/soubhiksanyal/FLAME_PyTorch/blob/master/requirements.txt
    # the requirenemnts needs to be fixed and we need to have chumpy installed
    # also python 3.10 is required with an "older" numpy version
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    path = Path(flame_dir) / "flame2023_no_jaw.pkl"
    with open(path, "rb") as f:
        return pickle.load(f, encoding="latin1")


def load_flame(flame_dir: str | Path):
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    path = Path(flame_dir) / "flame2023.pkl"
    with open(path, "rb") as f:
        return pickle.load(f, encoding="latin1")


# https://github.com/soubhiksanyal/FLAME_PyTorch/blob/master/flame_pytorch/flame.py
flame_dir = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame"
flame_model = load_flame_no_jaw(flame_dir)

# those are used in the pytorch flame example
faces = to_np(flame_model["f"])
shapedirs = to_np(flame_model["shapedirs"])
j_regressor = to_np(flame_model["J_regressor"])
posedirs = to_np(flame_model["posedirs"])
parents = to_np(flame_model["kintree_table"])
lbs_weights = to_np(flame_model["weights"])

# Falme Landmarks

The landmark file defines the barycentric embedding of 105 points of the Mediapipe mesh in the surface of FLAME.
In consists of three arrays: lmk_face_idx, lmk_b_coords, and landmark_indices.

- lmk_face_idx contains for every landmark the index of the FLAME triangle which each landmark is embedded into
- lmk_b_coords are the barycentric weights for each vertex of the triangles
- landmark_indices are the indices of the vertices of the Mediapipe mesh


In [ ]:
def load_flame_landmarks(flame_dir: str | Path):
    path = flame_dir = Path(flame_dir) / "mediapipe_landmark_embedding.npz"
    return np.load(path)


flame_landmarks = load_flame_landmarks(flame_dir)
print(list(flame_landmarks.keys()))
print()

print("lmk_face_idx:")
print(flame_landmarks["lmk_face_idx"][:5])
print(flame_landmarks["lmk_face_idx"].min())
print(flame_landmarks["lmk_face_idx"].max())
print(flame_landmarks["lmk_face_idx"].shape)
print()

print("lmk_b_coords:")
print(flame_landmarks["lmk_b_coords"][:5])
print(flame_landmarks["lmk_b_coords"].min())
print(flame_landmarks["lmk_b_coords"].max())
print(flame_landmarks["lmk_b_coords"].shape)
print()

print("landmark_indices:")
print(flame_landmarks["landmark_indices"][:5])
print(flame_landmarks["landmark_indices"].min())
print(flame_landmarks["landmark_indices"].max())
print(flame_landmarks["landmark_indices"].shape)

# FLAME Mask

Dictionary with vertex indices for different masks for the publicly available FLAME head model (https://flame.is.tue.mpg.de/).
See the gif for a visualization of all masks.


In [ ]:
def load_flame_masks(flame_dir: str | Path):
    path = Path(flame_dir) / "FLAME_masks.pkl"
    with open(path, "rb") as f:
        return pickle.load(f, encoding="latin1")


flame_masks = load_flame_masks(flame_dir)
print(list(flame_masks.keys()))
print()

print("eye_region:")
print(flame_masks["eye_region"][:5])
print(flame_masks["eye_region"].min())
print(flame_masks["eye_region"].max())
print(flame_masks["eye_region"].shape)